In [2]:
# Import packages in alphabetical order
import pandas as pd
import numpy as np
import tiktoken
import os
import openai
import duckdb
import owlready2
import sklearn

import gensim
import matplotlib.pyplot as plt
import nltk
import pickle
import re
import spacy
import string

# Import packages in alphabetical order to avoid duplicates
#from gensim.models import Word2Vec
#from nltk.corpus import stopwords
#from nltk.stem.porter import PorterStemmer
#from nltk.tokenize import word_tokenize
#from sklearn.cluster import KMeans
#from transformers import BertModel, BertTokenizer
from wordcloud import WordCloud

#Import libraries
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [3]:
#load the LLaMA model
import torch
import transformers

if torch.cuda.is_available():
    torch.set_default_device("cuda")
else:
    torch.set_default_device("cpu")
    
model = transformers.AutoModelForCausalLM.from_pretrained("microsoft/Orca-2-7b", device_map='auto')

# https://github.com/huggingface/transformers/issues/27132
# please use the slow tokenizer since fast and slow tokenizer produces different tokens
tokenizer = transformers.AutoTokenizer.from_pretrained(
        "microsoft/Orca-2-7b",
        use_fast=False,
    )

system_message = "You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior."
user_message = "How can you determine if a restaurant is popular among locals or mainly attracts tourists, and why might this information be useful?"

prompt = f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{user_message}<|im_end|>\n<|im_start|>assistant"

inputs = tokenizer(prompt, return_tensors='pt')
output_ids = model.generate(inputs["input_ids"],)
answer = tokenizer.batch_decode(output_ids)[0]

print(answer)

# This example continues showing how to add a second turn message by the user to the conversation
second_turn_user_message = "Give me a list of the key points of your first answer."

# we set add_special_tokens=False because we dont want to automatically add a bos_token between messages
second_turn_message_in_markup = f"\n<|im_start|>user\n{second_turn_user_message}<|im_end|>\n<|im_start|>assistant"
second_turn_tokens = tokenizer(second_turn_message_in_markup, return_tensors='pt', add_special_tokens=False)
second_turn_input = torch.cat([output_ids, second_turn_tokens['input_ids']], dim=1)

output_ids_2 = model.generate(second_turn_input,)
second_turn_answer = tokenizer.batch_decode(output_ids_2)[0]

print(second_turn_answer)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

2023-12-21 01:13:57.415406: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-21 01:13:57.415457: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-21 01:13:57.415489: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-21 01:13:57.424922: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


<s>  <|im_start|> system
You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior. <|im_end|> 
 <|im_start|> user
How can you determine if a restaurant is popular among locals or mainly attracts tourists, and why might this information be useful? <|im_end|> 
 <|im_start|> assistant
There are different ways to find out if a restaurant is popular among locals or mainly attracts tourists, and some possible reasons why this information might be useful are:

- If a restaurant has a high rating on online platforms like Yelp, TripAdvisor, or Google Reviews, it might indicate that it is popular among customers who have visited the area or are looking for recommendations from other travelers. However, these ratings might not reflect the preferences or tastes of the local population, who might have different expectations or standards for fo

In [3]:
with open('Datasets/grouped_sentences.pkl', 'rb') as f:
    grouped_sentences = pickle.load(f)

In [4]:
#Call grouped_sentences head
grouped_sentences.head()

,article_id,sentence,processed_sentence
0,1,Examining interdependence between product user...,examining interdependence between product user...
1,2,Information management as an enabler of knowle...,information management as an enabler of knowle...
2,3,Action Research and Critical Rationalisationis...,action research and critical rationalisationis...
3,4,Integration of Supply Chain Execution in B2B M...,integration of supply chain execution in marke...
4,5,Association for Information Systems AIS Electr...,association for information systems ais electr...


In [19]:
#Create sample text
sample_text = ["A large language model (LLM) is a large-scale language model notable for its ability to achieve general-purpose language understanding and generation. LLMs acquire these abilities by using massive amounts of data to learn billions of parameters during training and consuming large computational resources during their training and operation. LLMs are artificial neural networks (mainly transformers) and are (pre)trained using self-supervised learning and semi-supervised learning. As autoregressive language models, they work by taking an input text and repeatedly predicting the next token or word. Up to 2020, fine tuning was the only way a model could be adapted to be able to accomplish specific tasks. Larger sized models, such as GPT-3, however, can be prompt-engineered to achieve similar results. They are thought to acquire knowledge about syntax, semantics and ontology inherent in human language corpora, but also inaccuracies and biases present in the corpora.[5] Notable examples include OpenAI's GPT models (e.g., GPT-3.5 and GPT-4, used in ChatGPT), Google's PaLM (used in Bard), and Meta's LLaMA, as well as BLOOM, Ernie 3.0 Titan, and Anthropic's Claude 2."] #"A large language model (LLM) is a large-scale language model notable for its ability to achieve general-purpose language understanding and generation. LLMs acquire these abilities by using massive amounts of data to learn billions of parameters during training and consuming large computational resources during their training and operation.[1] LLMs are artificial neural networks (mainly transformers[2]) and are (pre)trained using self-supervised learning and semi-supervised learning. As autoregressive language models, they work by taking an input text and repeatedly predicting the next token or word.[3] Up to 2020, fine tuning was the only way a model could be adapted to be able to accomplish specific tasks. Larger sized models, such as GPT-3, however, can be prompt-engineered to achieve similar results.[4] They are thought to acquire knowledge about syntax, semantics and "ontology" inherent in human language corpora, but also inaccuracies and biases present in the corpora.[5] Notable examples include OpenAI's GPT models (e.g., GPT-3.5 and GPT-4, used in ChatGPT), Google's PaLM (used in Bard), and Meta's LLaMA, as well as BLOOM, Ernie 3.0 Titan, and Anthropic's Claude 2."

In [21]:
#Test the model with sample text of 180 words
system_message = "You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior."
user_message = "Please summarise the following text: " + sample_text[0]

prompt = f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{user_message}<|im_end|>\n<|im_start|>assistant"

inputs = tokenizer(prompt, return_tensors='pt')
output_ids = model.generate(inputs["input_ids"],)
answer = tokenizer.batch_decode(output_ids)[0]

print(answer)

<s>  <|im_start|> system
You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior. <|im_end|> 
 <|im_start|> user
Please summarise the following text: A large language model (LLM) is a large-scale language model notable for its ability to achieve general-purpose language understanding and generation. LLMs acquire these abilities by using massive amounts of data to learn billions of parameters during training and consuming large computational resources during their training and operation. LLMs are artificial neural networks (mainly transformers) and are (pre)trained using self-supervised learning and semi-supervised learning. As autoregressive language models, they work by taking an input text and repeatedly predicting the next token or word. Up to 2020, fine tuning was the only way a model could be adapted to be able to accomplish

This proves that the model is working, now trying with one row of data from the dataframe

In [22]:
#Test the model with one row
system_message = "You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior."
user_message = "Please summarise the following text: " + grouped_sentences['sentence'][0]

prompt = f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{user_message}<|im_end|>\n<|im_start|>assistant"

inputs = tokenizer(prompt, return_tensors='pt')
output_ids = model.generate(inputs["input_ids"],)
answer = tokenizer.batch_decode(output_ids)[0]

print(answer)

Token indices sequence length is longer than the specified maximum sequence length for this model (14555 > 4096). Running this sequence through the model will result in indexing errors
/home/roland/Projects/anhpham/ap-env/lib/python3.9/site-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 14555, but `max_length` is set to 4096. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 25.26 GiB. GPU 0 has a total capacty of 31.74 GiB of which 2.54 GiB is free. Process 514948 has 18.95 GiB memory in use. Including non-PyTorch memory, this process has 10.24 GiB memory in use. Of the allocated memory 7.71 GiB is allocated by PyTorch, and 1.65 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

The research text seems to be so long even with only one row. Trying to process with for loops and chunks

In [23]:
#Combining for loop for chunking and summarising
def generate_summary(article_text, max_chunk_size=1024, max_summary_length=100, num_beams=4):
    chunks = [article_text[i:i + max_chunk_size] for i in range(0, len(article_text), max_chunk_size)]
    summary_parts = []

    for chunk in chunks:
        
        system_message = "You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior."
        user_message = "Please summarise the following text: " + chunk

        prompt = f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{user_message}<|im_end|>\n<|im_start|>assistant"

        inputs = tokenizer(prompt, return_tensors='pt')
        output_ids = model.generate(inputs["input_ids"],)
        summary = tokenizer.batch_decode(output_ids)[0]

        summary_parts.append(summary)

    return " ".join(summary_parts)

In [24]:
#Trying chunks with one row
article_text = grouped_sentences['sentence'][0]

# Generate summary for the entire article
final_summary = generate_summary(article_text)

# Print the generated summary
print("\nGenerated Summary:")
print(final_summary)


Generated Summary:
<s>  <|im_start|> system
You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior. <|im_end|> 
 <|im_start|> user
Please summarise the following text: Examining interdependence between product users users and employees in online user communities : The role of employee - generated content Abstract. Firm - sponsored online user communities have become product innovation and support hubs of strategic importance to firms . Product users and host firm employees comprise the participants of firm - sponsored online user communities . The online user community provides a forum wherein the product users users and firm employees discuss questions , problems or issues resulting from the use of host firms ' products . Extant research on online user communities has largely focused on either product users users or employees 

It can be done, but the summarization is too fragmented. Increasing max chunk size and reducing summarization size.

In [21]:
#Combining for loop for chunking and summarising, fine tuning 2
def gen_summary(article_text, max_chunk_size=16384, max_summary_length=100, num_beams=4):
    chunks = [article_text[i:i + max_chunk_size] for i in range(0, len(article_text), max_chunk_size)]
    summary_parts = []

    for chunk in chunks:
        
        system_message = "You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior."
        user_message = "Please summarise the following text: " + chunk

        prompt = f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{user_message}<|im_end|>\n<|im_start|>assistant"

        inputs = tokenizer(prompt, return_tensors='pt')
        output_ids = model.generate(**inputs, do_sample=True, temperature=0.7, top_p=0.95, top_k=30, max_new_tokens=256)
        summary = tokenizer.batch_decode(output_ids)[0]
        summary_parts.append(summary)

    return " ".join(summary_parts)

In [22]:
#Trying chunks with one row
article_text = grouped_sentences['processed_sentence'][9999]

# Generate summary for the entire article
final_summary = gen_summary(article_text)

# Print the generated summary
print("\nGenerated Summary:")
print(final_summary)


Generated Summary:
<s>  <|im_start|> system
You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior. <|im_end|> 
 <|im_start|> user
Please summarise the following text: an agent mediated knowledge in motion model an agent mediated knowledge in motion model with millions invested in knowledge management knowledge management researchers and organizations are constantly investigating how firms can best organize their knowledge management processes to reap instrumental benefits yet most knowledge management research apart from being fragmented overemphasizes knowledge creation and draws little attention to key intermediaries in the knowledge management process the paper captures the specificity of agents as key players binding knowledge creation and knowledge application specifically the paper introduces conceptual process model tha

In [26]:
#Trying chunks with one row
article_text = grouped_sentences['processed_sentence'][9999]

# Generate summary for the entire article
final_summary = gen_summary(article_text)

# Print the generated summary
print("\nGenerated Summary:")
print(final_summary)


Generated Summary:

The text presents a new model of knowledge management that focuses on the role of agents as intermediaries in the process of creating and reusing knowledge. The model, called the agent mediated knowledge in motion model, views knowledge management as a cycle of transformations that agents facilitate. The text argues that this model is more realistic and useful than other approaches that neglect the agents' function.</s> 
The text is about how organizations can use agents (human or software) to manage their knowledge resources. It explains how agents can help create, transfer, apply, and disseminate knowledge in different stages of the process. It also describes an agent-mediated process that converts data into information, information into knowledge, and so on, until innovation is diffused back into data. The text shows how knowledge is a key resource that needs to be organized and reused.</s> 
The text is about the topic of knowledge management, which is how organ

In [19]:
#Combining for loop for chunking and summarising, fine tuning 2
def g_summary(article_text, max_chunk_size=16384, max_summary_length=100, num_beams=4):
    chunks = [article_text[i:i + max_chunk_size] for i in range(0, len(article_text), max_chunk_size)]
    summary_parts = []

    for chunk in chunks:
        
        system_message = "You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior."
        user_message = "Please summarise the following text: " + chunk

        prompt = f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{user_message}<|im_end|>\n<|im_start|>assistant"

        inputs = tokenizer(prompt, return_tensors='pt')
        output_ids = model.generate(**inputs, do_sample=True, temperature=0.7, top_p=0.95, top_k=30, max_new_tokens=256)
        summary = tokenizer.batch_decode(output_ids)[0].split("Summary:")[1].split("<</s>>")[0]

        summary_parts.append(summary)

    return " ".join(summary_parts)

In [4]:
def g_summary(article_text, max_chunk_size=16384, max_summary_length=100, num_beams=4):
    chunks = [article_text[i:i + max_chunk_size] for i in range(0, len(article_text), max_chunk_size)]
    summary_parts = []

    for chunk in chunks:
        system_message = "You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior."
        user_message = "Please summarise the following text: " + chunk

        prompt = f"system\n{system_message}\nuser\n{user_message}\nassistant"

        inputs = tokenizer(prompt, return_tensors='pt')
        output_ids = model.generate(**inputs, do_sample=True, temperature=0.7, top_p=0.95, top_k=30, max_new_tokens=256)

        decoded_output = tokenizer.batch_decode(output_ids)

        if decoded_output:
            summary_parts_chunk = decoded_output[0].split("Summary:")

            if len(summary_parts_chunk) > 1:
                summary_chunk = summary_parts_chunk[1].split("<</s>>")[0]
            else:
                # Handle the case where "Summary:" is not found in the chunk
                summary_chunk = "Default Summary"
        else:
            # Handle the case where decoded_output is empty
            summary_chunk = "No Summary Available"

        summary_parts.append(summary_chunk)

    return " ".join(summary_parts)


In [6]:
#Trying chunks with one row
article_text = grouped_sentences['processed_sentence'][9999]

# Generate summary for the entire article
final_summary = g_summary(article_text)

# Print the generated summary
print("\nGenerated Summary:")
print(final_summary)


Generated Summary:

The text summarizes a research paper that proposes an agent-mediated knowledge in motion model for knowledge management. The model views knowledge management as a series of knowledge transformations that are facilitated by agents as key players who bridge knowledge boundaries and enable knowledge creation, use, and innovation. The paper defines the model and the roles of different agents in the knowledge management process and suggests that it can help organizations use agents to create and apply knowledge for economic and competitive advantages. The paper also reviews the theoretical background and the research gap on agents in knowledge management.</s> 

The text is about how software agents and human agents can be used to manage and transform knowledge in organizations. It argues that software agents are not suitable for handling complexity, subtlety and multiple sources and threads of data, which are important for knowledge creation. It claims 

The text summar

The chunking with the manual fine tuning of the model looks good enough. Now continuing to random sampling the dataset for further analysis.
Also with the same standard in max_chunk_size=16834, do_sample=True, temperature=0.7, top_p=0.95, top_k=30, max_new_tokens=256.

In [7]:
#Load grouped_sentences_100 with random state from previous model
with open('Datasets/grouped_sentences_100.pkl', 'rb') as f:
    grouped_sentences_100 = pickle.load(f)

In [8]:
#Call grouped_sentences_100 head
grouped_sentences_100.head()

,article_id,sentence,processed_sentence
50374,50450,Association for Information Systems AIS Electr...,association for information systems ais electr...
13657,13667,Communications of the Association for Informat...,communications of the association for informat...
11449,11457,Effective organizational improvisation in info...,effective organizational improvisation in info...
46481,46549,Researching the Costs of Information Systems T...,researching the costs of information systems t...
30546,30587,Holographic Recommendations in Brick - and - M...,holographic recommendations in brick and morta...


In [10]:
#Create a copy of the grouped_sentences_100 dataframe as grouped_sentences_100_l
grouped_sentences_100_llama = grouped_sentences_100.copy()

In [24]:
#Create a for loop for every row of grouped_sentences_100_llama column 'processed_sentence' as article_text and save the result in a new column 'summary_llama'
for index, row in grouped_sentences_100_llama.iterrows():
    article_text = row['processed_sentence']
    final_summary = g_summary(article_text)
    grouped_sentences_100_llama.loc[index, 'summary'] = final_summary

#Save grouped_sentences_100 to pickle
with open('Datasets/grouped_sentences_100_llama.pkl', 'wb') as f:
    pickle.dump(grouped_sentences_100_llama, f)

#Call group_sentences_100_m head
grouped_sentences_100_llama.head()

,article_id,sentence,processed_sentence,summary
50374,50450,Association for Information Systems AIS Electr...,association for information systems ais electr...,\n\nThe article presents the research of the \...
13657,13667,Communications of the Association for Informat...,communications of the association for informat...,Default Summary Default Summary Default Summary
11449,11457,Effective organizational improvisation in info...,effective organizational improvisation in info...,Default Summary \nThe text is an abstract of a...
46481,46549,Researching the Costs of Information Systems T...,researching the costs of information systems t...,\n\nThe text summarizes a research paper that ...
30546,30587,Holographic Recommendations in Brick - and - M...,holographic recommendations in brick and morta...,\n\nThe text presents a research project that ...


In [5]:
#Load grouped_sentences_300 from pickle
with open('Datasets/grouped_sentences_300.pkl', 'rb') as f:
    grouped_sentences_300 = pickle.load(f)

In [6]:
#Set grouped_sentences_300 as a new variable grouped_sentences_300_m
grouped_sentences_300_l = grouped_sentences_300.copy()

In [7]:
#Create a for loop for every row of grouped_sentences_300_l column 'processed_sentence' as article_text and save the result in a new column 'summary_llama'
for index, row in grouped_sentences_300_l.iterrows():
    article_text = row['processed_sentence']
    final_summary = g_summary(article_text)
    grouped_sentences_300_l.loc[index, 'summary_llama'] = final_summary

#Save grouped_sentences_100 to pickle
with open('Datasets/grouped_sentences_300_l.pkl', 'wb') as f:
    pickle.dump(grouped_sentences_300_l, f)

#Call group_sentences_300_l head
grouped_sentences_300_l.head()

,article_id,sentence,processed_sentence,summary_llama
17899,17917,The firm 's continuance intentions to use inte...,the firm continuance intentions to use inter o...,\n \n\nThe text reviews the literature on the ...
9564,9570,The Philosopher 's Corner : Beyond Epistemolog...,the philosopher corner beyond epistemology and...,\n\nThe text is a summary of a philosophical d...
45598,45666,Interplay of Competition where ln Innovation w...,interplay of competition where ln innovation w...,\n\nThe text summarizes the main idea of the r...
39260,39308,The Impact of Crowdsourcing on Organisational ...,the impact of crowdsourcing on organisational ...,"\nThe paper examines how crowdmapping, a form ..."
23407,23440,Association for Information Systems Associatio...,association for information systems associatio...,Default Summary Default Summary Default Summar...
